In [6]:
import tkinter as tk
import os
import pandas as pd
import numpy as np
import re
import openpyxl
import base64
from tkinter import filedialog
from tkinter import messagebox
from io import BytesIO
from tkinter import PhotoImage

# Load user data
users_data = pd.read_csv("data/data.csv")
users_data_filtered = users_data[users_data["organizationalPerson.employeeID"] == os.getlogin()]
users_data_filtered_to_text = users_data_filtered["organizationalPerson.givenName"].to_string(index=False).strip()
loaded_data = None
selected_items = None
joined_items = None

# Set user name
user_name = os.getlogin()

def load_file():
    global loaded_data
    try:
        profit_center_listbox.delete(0, tk.END)
        existing_files_listbox.delete(0, tk.END)
        filepath = filedialog.askopenfilename(
            title="Open file",
            filetypes=(("Excel Files", "*.xlsx"), ("All Files", "*.*"))
        )
        if filepath:  # Check if a file path was selected
            file_label.config(text=os.path.basename(filepath)) # Update the label with the selected file
            main_data = pd.read_excel(filepath, sheet_name="Sheet1", skiprows=11)
            three_digit_values = main_data[main_data['Row Labels'].astype(str).str.match('^\d{3}$')]
            three_digit_list = [str(value) for value in three_digit_values['Row Labels']]
            for item in three_digit_list:
                profit_center_listbox.insert(tk.END, item)
            loaded_data = main_data

            if len(three_digit_list) == 0:
                messagebox.showwarning("Warning", "No Profit Centers found in the file.")

        existing_files_listbox.delete(0, tk.END)    
        output_folder = 'output'
        extracted_values = []
        if os.path.exists(output_folder):
            for file in os.listdir(output_folder):
                match = re.search(r'PC ([\d_]+)', file)
                if match:
                    extracted_values.append(match.group(1))
        for extracted_value in extracted_values:
            existing_files_listbox.insert(tk.END, extracted_value)

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}\nPlease load a valid file.")

def generate_files():
    
    errors = []
    # Check if all fields are filled
    if file_label.cget('text') == "No file loaded":
        errors.append("- No file has been loaded")
    if not document_date_entry.get():
        errors.append("- Document date is not filled in")
    if not posting_date_entry.get():
        errors.append("- Posting date is not filled in")
    if not reference_entry.get():
        errors.append("- Reference is not filled in")
    if len(profit_center_listbox.curselection()) == 0:
        errors.append("- No profit center has been selected")

    # If there are any errors, display a message with the list of problems
    if errors:
        messagebox.showwarning("Warning", "Please correct the following:\n" + "\n".join(errors))
        return  # Przerwanie funkcji, jeśli warunki nie są spełnione
    
    
    existing_files_listbox.delete(0, tk.END)
    global selected_items
    # Retrieve selected profit center values
    selected_items = [profit_center_listbox.get(i) for i in profit_center_listbox.curselection()]

    # Create filename
    if len(selected_items) == 1:
        filename = f"Wro LP 11 2023 PC {selected_items[0]} JE.xlsx"
        joined_items = '_'.join(selected_items)
    else:
        joined_items = '_'.join(selected_items)
        filename = f"Wro LP 11 2023 PC {joined_items} JE.xlsx"

    # Path to the 'output' folder
    output_folder = 'output'
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)  # Create 'output' folder if it does not exist

    prefix_name = reference_entry.get()
    prefix_name = prefix_name.replace("/", " ")
    prefix_name = prefix_name.replace("_", " ")
    
    city = prefix_name.split()[0]
    if city == "WAR":
        city = "W-wa"
    
    ppk_file = pd.read_excel("data/PPK.xlsx")

    def update_value(row):
        if pd.isna(row['Value']):
            return city
        elif row['Value'] == 'PPK':
            return f"PPK {city}"
        else:
            return row['Value']

    ppk_file['Value'] = ppk_file.apply(update_value, axis=1)
    
    
    
    df_file_name = file_label.cget('text')
    df = pd.read_excel(f"input/{df_file_name}", sheet_name="Sheet1", skiprows=11)
    df_rows_credit = pd.read_excel(f"input/{df_file_name}", sheet_name="Sheet1", skiprows=8)
    df_rows_debit = pd.read_excel(f"input/{df_file_name}", sheet_name="Sheet1", skiprows=7)
    
    # Ensure you convert all column names to strings
    df_rows_credit.columns = [str(col).split('.')[0] for col in df_rows_credit.columns]

    
    
    
    
    # Repeat the search for "Unnamed" columns in debit rows
    unnamed_index = None
    for i, col in enumerate(df_rows_credit.columns):
        if col.startswith("Unnamed"):
            unnamed_index = i
            break

    # If such a column exists, select columns up to that column (excluding it)
    if unnamed_index is not None:
        df_rows_credit = df_rows_credit.iloc[:, :unnamed_index]
    df_rows_credit = df_rows_credit.iloc[:, 1:] # Exclude the first column as before

    # Ensure all column names are converted to strings for debit rows
    df_rows_debit.columns = [str(col).split('.')[0] for col in df_rows_debit.columns]

    # Repeat the search for "Unnamed" columns in debit rows
    unnamed_index = None
    for i, col in enumerate(df_rows_debit.columns):
        if col.startswith("Unnamed"):
            unnamed_index = i
            break

    # Exclude the "Unnamed" column and following ones if found
    if unnamed_index is not None:
        df_rows_debit = df_rows_debit.iloc[:, :unnamed_index]
    df_rows_debit = df_rows_debit.iloc[:, 1:] # Exclude the first column as before

    
    # Lists of headers for credit and debit
    credit_headers_list = df_rows_credit.columns.tolist()
    debit_headers_list = df_rows_debit.columns.tolist()
    
    len_headers_list = len(credit_headers_list)
    
    # Remove "Unnamed" columns from the main dataframe as well
    unnamed_index = None
    for i, col in enumerate(df.columns):
        if col.startswith("Unnamed"):
            unnamed_index = i
            break

    if unnamed_index is not None:
        df = df.iloc[:, :unnamed_index]

    # Columns for output file
    output_file_columns = ["Debit Amount Doc Currency", "Credit Amount Doc Currency"]
    
    # Find the row index containing 'Grand Total' in 'Row Labels' column
    grand_total_index = df.index[df['Row Labels'] == 'Grand Total'].tolist()

    # Exclude rows from 'Grand Total' downwards if present
    if grand_total_index:
        df = df.iloc[:grand_total_index[0]]
    
    # Dictionary to store new DataFrames
    dfs = {}

    # Temporary variable to store current profit center
    current_profit_center = None

    for i, row in df.iterrows():
        # Check if the row contains profit center
        if len(str(row['Row Labels'])) == 3 and str(row['Row Labels']).isdigit():
            current_profit_center = row['Row Labels']
            # Create new DataFrame for this profit center
            dfs[current_profit_center] = pd.DataFrame(columns=df.columns)
        elif current_profit_center is not None:
            # Add row to the corresponding DataFrame
            dfs[current_profit_center] = dfs[current_profit_center].append(row)
    for profit_center in dfs:
        # Transpose dataframes to switch rows and columns
        dfs[profit_center] = dfs[profit_center].transpose()

    for profit_center in dfs:
        # Set the first row as heade
        dfs[profit_center].columns = dfs[profit_center].iloc[0]

        # Remove the first row from data
        dfs[profit_center] = dfs[profit_center].iloc[1:]
    
    # Copy DataFrames for backup
    dfsB = {profit_center: df.copy(deep=True) for profit_center, df in dfs.items()}   

    for profit_center, df in dfs.items():
        # Ensure list has appropriate length
        if len(credit_headers_list) == len(df):
            # Add new column with values from list
            df['New Column'] = credit_headers_list
        else:
            # Fill missing values with NaN or zeros if lengths do not match
            df['New Column'] = credit_headers_list[:len(df)] + [np.nan] * (len(df) - len(credit_headers_list))

        # Update DataFrame in the dictionary
        dfs[profit_center] = df

    for profit_center, df in dfsB.items():
        # Ensure list has appropriate length for debit headers
        if len(debit_headers_list) == len(df):
            df['New Column'] = debit_headers_list
        else:
            df['New Column'] = debit_headers_list[:len(df)] + [np.nan] * (len(df) - len(debit_headers_list))

        # Update DataFrame in the dictionary for backup
        dfsB[profit_center] = df
    
    # Combine corresponding DataFrames from dfs and dfsB into a new dictionary
    dfsConcat = {}

    for profit_center in dfs:
        if profit_center in dfsB:
            # Use pd.concat to combine DataFrames along row axis (axis=0)
            dfsConcat[profit_center] = pd.concat([dfsB[profit_center], dfs[profit_center]], axis=0).reset_index(drop=True)
        else:
            # If no corresponding DataFrame in dfsB, just copy dfs
            dfsConcat[profit_center] = dfsB[profit_center].copy(deep=True)

    # Combine corresponding DataFrames from dfs and dfsB into a new dictionary
    dfsConcatNew = {}

    for key, df in dfsConcat.items():
        if str(key) in selected_items:
            # List for storing individual rows
            all_rows = []

            # Iterate through all columns except "New Column"
            for col in df.columns[:-1]:  # Skip the last "New Column"
                # Iterate through all rows in the current column
                for index, value in df[col].iteritems():
                    # Create a single row with data
                    new_row = {
                        'DataFrame Key': key,  # Dodanie klucza DataFrame jako kolumny
                        'test': value,  # Wartość z bieżącej kolumny
                        'test2': col,   # Nazwa bieżącej kolumny
                        'New Column': df.at[index, 'New Column']  # Wartość z kolumny "New Column"
                    }
                    # Add the new row to the list of all rows
                    all_rows.append(new_row)

            # Create a new DataFrame from all rows
            new_df = pd.DataFrame(all_rows)
            
            # Round up the division to ensure enough repetitions
            repeat_times = -(-len(new_df) // len(ppk_file))  # Zaokrąglenie w górę podziału
            
            # Repeat the values from PPK file to match the length of new_df
            values_to_add = np.tile(ppk_file['Value'], repeat_times)[:len(new_df)]
            
            # Add values to the new DataFrame
            new_df['Value'] = values_to_add

            # Remove rows where the value in 'test' column is 0
            new_df_filtered = new_df[new_df['test'] != 0]

            # Add the filtered DataFrame to the dictionary
            dfsConcatNew[key] = new_df_filtered

    # dfsConcatNew now contains filtered DataFrames with an additional 'DataFrame Key' column

    dfsConcatNewDivided = {}

    for key, df in dfsConcatNew.items():
        # Initialize new columns with zeros
        df['Debit Amount Doc Currency'] = 0
        df['Credit Amount Doc Currency'] = 0

        # Iterate through DataFrame rows
        for i, row in df.iterrows():
            # Check which "group" of rows we are in and assign values to the corresponding column
            if (i // len_headers_list) % 2 == 0:
                if row['test'] >= 0:
                    df.at[i, 'Debit Amount Doc Currency'] = row['test']
                else:
                    df.at[i, 'Credit Amount Doc Currency'] = -row['test']  # Ujemne wartości jako dodatnie w kredycie
            else:
                if row['test'] >= 0:
                    df.at[i, 'Credit Amount Doc Currency'] = row['test']
                else:
                    df.at[i, 'Debit Amount Doc Currency'] = -row['test']  # Ujemne wartości jako dodatnie w debecie

        # Update DataFrame in the dictionary
        dfsConcatNewDivided[key] = df
    
    # Merge divided DataFrames into one and assign an index
    dfsConcatNewMerged = pd.concat(dfsConcatNewDivided.values(), ignore_index=True)
    dfsConcatNewMerged['Index'] = range(1, len(dfsConcatNewMerged) + 1)
    
    # Path to the layout file
    layout_path = os.path.join("data", "Wro LP.xlsx")
    
    # Full path to the file
    file_path = os.path.join(output_folder, filename)

    
    # Logic for creating the content of the XLSX file
    workbook = openpyxl.load_workbook(layout_path)
    sheet = workbook['Test JE template to post']
    row_start = 13
    
    # Calculate the last row to be filled
    last_row = row_start + len(dfsConcatNewMerged) - 1
    
    # Replace zeros with NaN in specific columns to avoid displaying them in Excel
    dfsConcatNewMerged["Debit Amount Doc Currency"] = dfsConcatNewMerged["Debit Amount Doc Currency"].replace(0, np.nan)
    dfsConcatNewMerged["Credit Amount Doc Currency"] = dfsConcatNewMerged["Credit Amount Doc Currency"].replace(0, np.nan)
    
    # Fill the sheet with data from dfsConcatNewMerged
    for i, row in dfsConcatNewMerged.iterrows():
        # Add data to the sheet
        sheet[f'A{row_start + i}'] = row['Index']
        sheet[f'B{row_start + i}'] = row['New Column']
        sheet[f'C{row_start + i}'] = row['Debit Amount Doc Currency']
        sheet[f'D{row_start + i}'] = row['Credit Amount Doc Currency']
        sheet[f'H{row_start + i}'] = row['test2']
        sheet[f'L{row_start + i}'] = row['DataFrame Key']
        sheet[f'K{row_start + i}'] = row['Value']
        sheet[f'J{row_start + i}'] = reference_entry.get()
    
    # Insert formulas to calculate the sum of debit and credit amounts in Excel
    sheet[f'C{last_row + 1}'] = f'=SUM(C{row_start}:C{last_row})' 
    sheet[f'D{last_row + 1}'] = f'=SUM(D{row_start}:D{last_row})'
    sheet['I1'] = f'=SUM(C{row_start}:C{last_row})' # Total Debit in the document header
    sheet['I2'] = f'=SUM(D{row_start}:D{last_row})' # Total Credit in the document header
    
    # Set document, posting dates, and reference based on user input
    sheet['C1'] = document_date_entry.get()
    sheet['C2'] = posting_date_entry.get()
    sheet['C3'] = reference_entry.get()
    sheet['C4'] = reference_entry.get()

    # Construct final filename based on user input and selected profit centers
    filename = f"{prefix_name} PC {joined_items} JE.xlsx"

    output_path = os.path.join("output", filename)
    
    # Save the workbook to the specified path
    workbook.save(output_path)
    
    # Clear the listbox displaying existing files
    existing_files_listbox.delete(0, tk.END)
    output_folder = 'output'
    extracted_values = []
    # Reload and display newly created or existing files in the output folder
    if os.path.exists(output_folder):
        for file in os.listdir(output_folder):
            match = re.search(r'PC ([\d_]+)', file)
            if match:
                extracted_values.append(match.group(1))
    for extracted_value in extracted_values:
            existing_files_listbox.insert(tk.END, extracted_value)

            
# Function placeholder for checking existing files (not implemented)
def check_existing_files():
    # Tutaj logika sprawdzania istniejących plików
    pass

# Initial setup for the main application window
root = tk.Tk()
root.withdraw() # Hide the main window immediately

def on_close():
    """Handles the event when the user attempts to close the Toplevel window."""
    top.destroy()  # Destroy the Toplevel window first
    root.destroy() # Then destroy the main window, which exits the application

# Setup for the Toplevel window, which will serve as the main application window
top = tk.Toplevel()
top.protocol("WM_DELETE_WINDOW", on_close)
top.title("Excel Profit Center app")
width = 380
height = 400

# Load and display the info image
with open("data/info.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode()

def show_info_image():
    """Displays an information image in a new Toplevel window."""
    info_window = tk.Toplevel(top)
    info_window.title("Info")

    image_data = base64.b64decode(encoded_string)
    image = PhotoImage(data=image_data)

    img_label = tk.Label(info_window, image=image)
    img_label.image = image  # Keep a reference to avoid garbage collection
    img_label.pack()
  
    
    
    
# Calculate and set the position for the application window to center it on the screen
screen_width = top.winfo_screenwidth() # Get the width of the screen
screen_height = top.winfo_screenheight() # Get the height of the screen
x_position = int((screen_width/2) - (width/2)) # Calculate the X position to center the window
y_position = int((screen_height/2) - (height/2)) # Calculate the Y position to center the window
top.geometry(f"{width}x{height}+{x_position}+{y_position}") # Set the geometry to center the window

# Labels for the entry fields
labels_texts = ['Document Date', 'Posting Date', 'Reference']
entries = {} # Dictionary to store the entry widgets

# Add a welcome label
welcome_label = tk.Label(top, text=f"Witaj, {users_data_filtered_to_text}!")
welcome_label.grid(row=0, column=0, columnspan=3, pady=(10, 10), sticky='ew')

# Button to show info
info_button = tk.Button(top, text="Info", command=show_info_image)
info_button.grid(row=0, column=2, padx=10, pady=(10, 0), sticky='ne')


# Frame for file label and load button
file_frame = tk.Frame(top)
file_frame.grid(row=1, column=0, columnspan=3, padx=10, sticky='ew')

# Label for the selected file
file_label = tk.Label(file_frame, text="No file loaded", anchor='w')
file_label.pack(side=tk.LEFT, fill=tk.X, expand=True)

# Button to load a file
load_button = tk.Button(file_frame, text="Load File", command=load_file)
load_button.pack(side=tk.RIGHT)

# Label and entry field for Document Date
document_date_label = tk.Label(top, text="Document Date", width=15)
document_date_label.grid(row=2, column=0, padx=10, pady=(10, 0), sticky='e')
document_date_entry = tk.Entry(top, width=15)
document_date_entry.grid(row=2, column=1, padx=10, pady=(10, 0), sticky='ew')

# Label and entry field for Posting Date
posting_date_label = tk.Label(top, text="Posting Date", width=15)
posting_date_label.grid(row=3, column=0, padx=10, pady=(10, 0), sticky='e')
posting_date_entry = tk.Entry(top, width=15)
posting_date_entry.grid(row=3, column=1, padx=10, pady=(10, 0), sticky='ew')

# Label and entry field for Reference
reference_label = tk.Label(top, text="Reference", width=15)
reference_label.grid(row=4, column=0, padx=10, pady=(10, 0), sticky='e')
reference_entry = tk.Entry(top, width=15)
reference_entry.grid(row=4, column=1, padx=10, pady=(10, 0), sticky='ew')

# Label for the Profit Center Listbox
pc_label = tk.Label(top, text="Profit Center List")
pc_label.grid(row=5, column=0, padx=10, pady=(10, 0))

# Listbox for Profit Centers
profit_center_listbox = tk.Listbox(top, selectmode='multiple', width=15)
profit_center_listbox.grid(row=6, column=0, padx=10, pady=10, sticky='nsew')

# Label for the Existing Files Listbox
done_pc_label = tk.Label(top, text="Done Profit Center")
done_pc_label.grid(row=5, column=2, padx=10, pady=(10, 0))

# Listbox for existing Profit Center files
existing_files_listbox = tk.Listbox(top, width=15)
existing_files_listbox.grid(row=6, column=2, padx=10, pady=10, sticky='nsew')

# Button to generate files
generate_button = tk.Button(top, text="Generate Files", command=generate_files)
generate_button.grid(row=6, column=1, padx=10, pady=80, sticky='nsew')

# Start the main loop
top.mainloop()


C:\Users\30791552\AppData\Local\Temp\1\ipykernel_17440\303659736.py:194: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfs[current_profit_center] = dfs[current_profit_center].append(row)
C:\Users\30791552\AppData\Local\Temp\1\ipykernel_17440\303659736.py:194: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfs[current_profit_center] = dfs[current_profit_center].append(row)
C:\Users\30791552\AppData\Local\Temp\1\ipykernel_17440\303659736.py:194: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfs[current_profit_center] = dfs[current_profit_center].append(row)
C:\Users\30791552\AppData\Local\Temp\1\ipykernel_17440\303659736.py:194: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ve